In [1]:
from torch.utils.data import DataLoader, Subset
from torchvision import models,transforms
import pandas as pd
import numpy as np
import os
from PIL import Image
from typing import Tuple, List
import torch
from torch import optim,nn
from torch.autograd import Variable
from tqdm import tqdm
import matplotlib.pyplot as plt

from dataset import MaskDataset, AddGaussianNoise
from models import get_pre_trained

data_dir = '../../input/data/train'

In [2]:
os.getcwd()

'/opt/ml/level1-image-classification-level1-recsys-08/code'

In [3]:
# model_name = 'densenet161'
model_name = 'resnet50'
num_classes = 18
# Initialize the model for this run
model = get_pre_trained(model_name, feature_extract = False, pretrained = True)
# Define the device:
device = torch.device('cuda:0')
# Put the model on the device:
model = model.to(device)

In [4]:
norm_mean = (0.485, 0.456, 0.406)
norm_std = (0.229, 0.224, 0.225)

train_transform = transforms.Compose([transforms.RandomRotation(degrees = 15),
                                      transforms.ColorJitter(brightness=0.1, contrast=0.1, hue=0.1),
                                      transforms.RandomVerticalFlip(p=0.5),
                                      transforms.ToTensor(),
                                      transforms.Normalize(norm_mean, norm_std),
#                                       AddGaussianNoise()
                                     ])
val_transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(norm_mean, norm_std)])

dataset = MaskDataset(data_dir=data_dir,transforms=train_transform, adj_csv = True, val_ratio=0.1, up_sampling = 3)
batch_size = 16
train_set, val_set = dataset.split_dataset()
use_cuda = torch.cuda.is_available()

train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=use_cuda,
    drop_last=True,
)

val_loader = DataLoader(
    val_set,
    batch_size=batch_size,
    shuffle=False,
    pin_memory=use_cuda,
    drop_last=True,
)

upsamling starts ...
Data split completed: val_ratio=0.1
n_train=26064, n_val=2895


In [5]:
# this function is used during training process, to calculation the loss and accuracy
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [6]:
def train(model, train_loader, criterion, optimizer, epoch):
    save_dir = os.path.join(os.getcwd(), 'save')
    model.train()
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    curr_iter = (epoch - 1) * len(train_loader)
    
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    
    for i, data in enumerate(train_loader):
        images, labels = data
        N = images.size(0)
        # print('image shape:',images.size(0), 'label shape',labels.size(0))
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        prediction = outputs.max(1, keepdim=True)[1]
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
        train_loss.update(loss.item())
        curr_iter += 1
        if (i + 1) % 200 == 0:
            print('[epoch %d], [iter %d / %d], [train loss %.5f], [train acc %.5f]' % (
                epoch, i + 1, len(train_loader), train_loss.avg, train_acc.avg))
            total_loss_train.append(train_loss.avg)
            total_acc_train.append(train_acc.avg)
    return train_loss.avg, train_acc.avg

In [7]:
def validate(model, val_loader, criterion, optimizer, epoch):
    model.eval()
    val_loss = AverageMeter()
    val_acc = AverageMeter()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            images, labels = data
            N = images.size(0)
            images = Variable(images).to(device)
            labels = Variable(labels).to(device)

            outputs = model(images)
            prediction = outputs.max(1, keepdim=True)[1]

            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)

            val_loss.update(criterion(outputs, labels).item())

    print('------------------------------------------------------------')
    print('[epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss.avg, val_acc.avg))
    print('------------------------------------------------------------')
    return val_loss.avg, val_acc.avg

In [ ]:
MODEL_PATH = '../model' + '/' + model_name
os.makedirs(MODEL_PATH, exist_ok=True)
epoch_num = 12
best_val_acc = 0
total_loss_train, total_acc_train = [],[]
total_loss_val, total_acc_val = [],[]
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss().to(device)
for epoch in tqdm(range(1, epoch_num+1)):
    loss_train, acc_train = train(model, train_loader, criterion, optimizer, epoch)
    loss_val, acc_val = validate(model, val_loader, criterion, optimizer, epoch)
    total_loss_val.append(loss_val)
    total_acc_val.append(acc_val)
    if acc_val > best_val_acc:
        best_val_acc = acc_val
        torch.save(model.state_dict(), os.path.join(MODEL_PATH, f"{model_name}_best.pt"))
        print('*****************************************************')
        print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, loss_val, acc_val))
        print('*****************************************************')

  0%|          | 0/12 [00:00<?, ?it/s]

[epoch 1], [iter 200 / 1629], [train loss 2.43765], [train acc 0.25594]
[epoch 1], [iter 400 / 1629], [train loss 2.34316], [train acc 0.27406]
[epoch 1], [iter 600 / 1629], [train loss 2.22970], [train acc 0.29844]
[epoch 1], [iter 800 / 1629], [train loss 2.08605], [train acc 0.33172]
[epoch 1], [iter 1000 / 1629], [train loss 1.95078], [train acc 0.36538]
[epoch 1], [iter 1200 / 1629], [train loss 1.83394], [train acc 0.39854]
[epoch 1], [iter 1400 / 1629], [train loss 1.72570], [train acc 0.42933]
[epoch 1], [iter 1600 / 1629], [train loss 1.63297], [train acc 0.45621]
------------------------------------------------------------
[epoch 1], [val loss 0.93630], [val acc 0.65278]
------------------------------------------------------------


  8%|▊         | 1/12 [25:14<4:37:42, 1514.74s/it]

*****************************************************
best record: [epoch 1], [val loss 0.93630], [val acc 0.65278]
*****************************************************


In [ ]:
fig = plt.figure(num = 2, figsize = (12,6))
fig1 = fig.add_subplot(2,1,1)
fig2 = fig.add_subplot(2,1,2)
fig1.plot(total_loss_train, label = 'training loss')
fig1.plot(total_acc_train, label = 'training accuracy')
fig1.legend()
fig2.plot(total_loss_val, label = 'validation loss')
fig2.plot(total_acc_val, label = 'validation accuracy')
fig2.legend()
plt.show()